In [1]:
from pathlib import Path
from asim.dataset.scene.arrow_scene import ArrowScene
from asim.common.visualization.matplotlib.plots import plot_scene_at_iteration


# log_name = "2021.06.07.11.59.52_veh-35_00095_00555"
# log_file = Path(f"/home/daniel/asim_workspace/data/nuplan_val/{log_name}.arrow")
log_name = "_Rep0_bench2drive220_route0_06_12_20_48_54"
log_file = Path(f"/home/daniel/asim_workspace/data/carla/{log_name}.arrow")
scene = ArrowScene(log_file)

plot_scene_at_iteration(scene, iteration=10)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import traceback
from asim.common.visualization.matplotlib.plots import render_scene_as_mp4

output_path = Path("/home/daniel/asim_logs_videos")
# render_scene_as_mp4(scene, output_path, fps=30, end_idx=10000, step=5, dpi=100)

In [3]:
# Create an mp4 animation with a specific FPS
import traceback
from asim.common.visualization.matplotlib.plots import render_scene_as_mp4


output_path = Path("/home/daniel/asim_logs_videos")
log_path = Path("/home/daniel/asim_workspace/data/nuplan_mini_val")
for log_file in log_path.iterdir():
    scene = ArrowScene(log_file)
    try:
        render_scene_as_mp4(scene, output_path, fps=30, end_idx=None, step=5)
    except Exception as e:
        traceback.print_exc()
        erroneous_file = output_path / f"{log_name}.mp4"
        if erroneous_file.exists():
            erroneous_file.unlink()
    del scene
    # break

Rendering 2021.06.08.14.35.24_veh-26_02555_03004 as MP4: 900it [04:21,  3.44it/s]                         
Rendering 2021.10.05.07.10.04_veh-52_01442_01802 as MP4: 721it [02:24,  4.97it/s]                         
Rendering 2021.07.24.23.50.16_veh-17_01696_02071 as MP4: 751it [03:32,  3.53it/s]                         
Rendering 2021.06.08.16.31.33_veh-38_01589_02072 as MP4: 967it [05:34,  2.89it/s]                         
Rendering 2021.06.08.12.54.54_veh-26_04262_04732 as MP4: 942it [04:24,  3.56it/s]                         
Rendering 2021.06.07.12.54.00_veh-35_01843_02314 as MP4: 943it [04:17,  3.67it/s]                         
Rendering 2021.08.09.17.55.59_veh-28_00021_00307 as MP4: 573it [01:52,  5.12it/s]                         
Rendering 2021.08.24.13.12.55_veh-45_00386_00472 as MP4: 173it [00:36,  4.74it/s]                         
Rendering 2021.06.07.18.53.26_veh-26_00005_00427 as MP4: 845it [04:54,  2.87it/s]                         
Rendering 2021.07.24.20.37.45_veh-17_